In [ ]:
#Installations
!pip install pandas dash plotly

In [4]:
#Imports
import pandas as pd
import matplotlib as plt
import numpy as np
import requests

import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

<h4><strong>Step 1 : Loading Datasets</strong></h4>

We have three datasets, 
<ul>
    <li>Annual Greenhouse gas emissions - Tidy format</li>
    <li>Carbon tax status</li>
    <li>Contribution to Temperature rise by Greenhouse Gases</li>
</ul>

In [5]:
#Dataset 1 : Combined dataset of Annual Emissions
df = pd.read_csv('tidy_df.csv')
df.head()

,Entity,Code,Year,Gas Type,Emissions
0,Afghanistan,AFG,1850,N₂O Emissions,224421.52
1,Afghanistan,AFG,1851,N₂O Emissions,229096.17
2,Afghanistan,AFG,1852,N₂O Emissions,233650.48
3,Afghanistan,AFG,1853,N₂O Emissions,238009.98
4,Afghanistan,AFG,1854,N₂O Emissions,242100.23


In [6]:
df['Gas Type'].unique()

array(['N₂O Emissions', 'Methane Emissions', 'CO₂ Emissions',
       'Overall Emissions'], dtype=object)

In [7]:
#Dataset 2 : Carbon tax status
df_tax = pd.read_csv("https://ourworldindata.org/grapher/carbon-tax-instruments.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
df_tax.head()

,Entity,Code,Year,tax
0,Afghanistan,AFG,1989,No carbon tax
1,Afghanistan,AFG,1990,No carbon tax
2,Afghanistan,AFG,1991,No carbon tax
3,Afghanistan,AFG,1992,No carbon tax
4,Afghanistan,AFG,1993,No carbon tax


In [8]:
df_tax['tax'].unique()

array(['No carbon tax', 'Has a carbon tax',
       'Has a carbon tax only at a sub-national level'], dtype=object)

In [9]:
#Dataset 3 : Contributions to Temperature
df_temp = pd.read_csv("https://ourworldindata.org/grapher/contribution-temp-rise-degrees.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
df_temp.head()

,Entity,Code,Year,temperature_response_ghg_total
0,Afghanistan,AFG,1851,0.000003
1,Afghanistan,AFG,1852,0.000006
2,Afghanistan,AFG,1853,0.000009
3,Afghanistan,AFG,1854,0.000012
4,Afghanistan,AFG,1855,0.000015


In [10]:
print("Maximum Temperature Change: ", df_temp['temperature_response_ghg_total'].max())
print("Minimum Temperature Change: ", df_temp['temperature_response_ghg_total'].min())

Maximum Temperature Change:  1.6683129
Minimum Temperature Change:  -0.00056490005


<h4><strong>Step 2 : Data Preprocessing</strong></h4>

In [11]:
df_temp.rename(columns={'temperature_response_ghg_total': 'Temperature Change'}, inplace=True)

One of the datasets loaded above have records from 1950 to 2023 while the others are from 1850. So to maintain uniformity, we will be considering the year scale of 1950 to 2023 for all the datasets.

In [12]:
df=df[df['Year'] >= 1950]

<strong>Dataset Transformation : </strong>The following code transforms the data from a tidy format to a hybrid format(tidy + wide) that's optimized for interactive visualization because:
<ul>
    <li>Tidy Format : Switching between gas types is more efficient when all emissions data is available in the same row. This structure ensures data integrity.</li>
    <li>Wide Format : Plotly tooltips work better with wide-format data where each row contains all relevant information for a single point. This structure is for visualization needs.</li>
</ul>
The composite key (Code, Year) ensures unique identification of records.

In [13]:
# Group the DataFrame by 'Code' and 'Year'
grouped_df = df.groupby(['Code', 'Year'])

# Precompute the emissions and dictionaries separately
co2_emissions_dict = grouped_df.apply(lambda x: x[x['Gas Type'] == 'CO₂ Emissions']['Emissions'].values[0] if not x[x['Gas Type'] == 'CO₂ Emissions'].empty else np.nan, include_groups=False).to_dict()
n2o_emissions_dict = grouped_df.apply(lambda x: x[x['Gas Type'] == 'N₂O Emissions']['Emissions'].values[0] if not x[x['Gas Type'] == 'N₂O Emissions'].empty else np.nan, include_groups=False).to_dict()
methane_emissions_dict = grouped_df.apply(lambda x: x[x['Gas Type'] == 'Methane Emissions']['Emissions'].values[0] if not x[x['Gas Type'] == 'Methane Emissions'].empty else np.nan, include_groups=False).to_dict()
total_emissions_dict = grouped_df.apply(lambda x: x[x['Gas Type'] == 'Overall Emissions']['Emissions'].values[0] if not x[x['Gas Type'] == 'Overall Emissions'].empty else np.nan, include_groups=False).to_dict()

def calculate_contributions_and_emissions(row):
    # Get the emission values from the dictionaries
    co2_emission = co2_emissions_dict.get((row['Code'], row['Year']), np.nan)
    n2o_emission = n2o_emissions_dict.get((row['Code'], row['Year']), np.nan)
    methane_emission = methane_emissions_dict.get((row['Code'], row['Year']), np.nan)
    total_emission = total_emissions_dict.get((row['Code'], row['Year']), np.nan)
    
    # Initialize all values as NaN
    result = {
        'CO₂ Contribution (%)': np.nan,
        'N₂O Contribution (%)': np.nan,
        'Methane Contribution (%)': np.nan,
        'CO₂ Emissions': np.nan,
        'N₂O Emissions': np.nan,
        'Methane Emissions': np.nan
    }
    
    # Set values based on Gas Type
    if row['Gas Type'] == 'Overall Emissions':
        # Calculate contributions only for Overall Emissions
        if not np.isnan(total_emission) and not np.isnan(co2_emission) and not np.isnan(n2o_emission) and not np.isnan(methane_emission):
            result['CO₂ Contribution (%)'] = (co2_emission / total_emission) * 100
            result['N₂O Contribution (%)'] = (n2o_emission / total_emission) * 100
            result['Methane Contribution (%)'] = (methane_emission / total_emission) * 100
    elif row['Gas Type'] == 'CO₂ Emissions':
        result['CO₂ Emissions'] = row['Emissions']
    elif row['Gas Type'] == 'N₂O Emissions':
        result['N₂O Emissions'] = row['Emissions']
    elif row['Gas Type'] == 'Methane Emissions':
        result['Methane Emissions'] = row['Emissions']
    
    return pd.Series(result)

# Apply the function to create/update all columns at once
contribution_emission_cols = [
    'CO₂ Contribution (%)', 'N₂O Contribution (%)', 'Methane Contribution (%)',
    'CO₂ Emissions', 'N₂O Emissions', 'Methane Emissions'
]
df[contribution_emission_cols] = df.apply(calculate_contributions_and_emissions, axis=1)

# Check the columns to verify the results
print(df.columns)

Index(['Entity', 'Code', 'Year', 'Gas Type', 'Emissions',
       'CO₂ Contribution (%)', 'N₂O Contribution (%)',
       'Methane Contribution (%)', 'CO₂ Emissions', 'N₂O Emissions',
       'Methane Emissions'],
      dtype='object')


In [14]:
df.tail()

,Entity,Code,Year,Gas Type,Emissions,CO₂ Contribution (%),N₂O Contribution (%),Methane Contribution (%),CO₂ Emissions,N₂O Emissions,Methane Emissions
164947,Zimbabwe,ZWE,2019,Overall Emissions,34347856.0,47.442359,14.835511,37.722130,NaN,NaN,NaN
164948,Zimbabwe,ZWE,2020,Overall Emissions,31322906.0,45.031655,14.603926,40.364422,NaN,NaN,NaN
164949,Zimbabwe,ZWE,2021,Overall Emissions,33549390.0,46.633393,14.480087,38.886519,NaN,NaN,NaN
164950,Zimbabwe,ZWE,2022,Overall Emissions,33772416.0,47.290777,14.375844,38.333378,NaN,NaN,NaN
164951,Zimbabwe,ZWE,2023,Overall Emissions,33955148.0,48.629896,13.718825,37.651278,NaN,NaN,NaN


The final format is a hybrid format where :
<ol>
    <li>When Gas Type is Overall Emissions : we would have values for 'CO₂ Contribution (%)', 'N₂O Contribution (%)',
       'Methane Contribution (%)' and 'CO₂ Emissions', 'N₂O Emissions',
       'Methane Emissions' these would be null.</li>
    <li>When Gas Type is 'CO₂ Emissions' we would only have a value in that column and rest would be NaN. The same goes for other types.</li>
</ol>

<h4><strong>Step 3 : App Setup</strong></h4>

Plotly Dash was chosen for this project for several key advantages:
<ul>
    <li>Interactive Visualization: Dash enables creation of interactive web applications with dynamic maps and charts without requiring separate frontend/backend development.</li>
    <li>Python Integration: Allows seamless integration with pandas dataframes and other Python data analysis tools.</li>
    <li>Real-time Updates: Supports callback functions for real-time data updates and user interactions.</li>
</ul>

The setup creates a foundation for building an interactive emissions visualization dashboard where users can explore and analyze global emissions data through maps and other visual components.

In [15]:
# App setup
app = dash.Dash(__name__,suppress_callback_exceptions=True)
app.title = "Global Emissions Map"

<h4><strong>Step 4 : App Layout</strong></h4>
The app layout in Plotly Dash defines the structure and organization of the web application's interface through the following key functions. It is like designing a HTML page of a website.

The layout serves as the blueprint for how users will interact with the emissions data visualization, ensuring a logical flow and intuitive user experience.

In [16]:
app.layout = html.Div([
    #Top Heading of the plot
    html.H1("Greenhouse Gases - Global emissions over time", style={'text-align': 'center','margin-bottom': '2%','margin-top': '4%','margin-left':'6%'}),

    #Side Menu Div
    html.Div([
        #Collapsible button for the side menu
        html.Div([
            html.Button('☰', id='collapse-button', n_clicks=0,
                        style={'font-size': '30px', 'background': 'none', 'border': 'none', 'cursor': 'pointer','display':'none'}),
        ], style={'margin-bottom': '10px'}),

        #Side Menu contents
        html.Div([
            #Category
            html.H4("Category", style={'margin-bottom': "15px", "font-size": "18px"}),
            html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '15px'}),
            dcc.RadioItems(
                id='display-mode-radio',
                options=[
                    {'label': "Emissions", "value": "Emissions"},
                    {'label': "Carbon Tax Status", "value": "Carbon Tax Status"},
                    {'label': "Temperature Change", "value": "Temperature Change"}
                ],
                value="Emissions",  # Default selection
                labelStyle={"display": "block", "margin": "15px 0", "color": "black", "font-size": "15px"}
            ),

            # Emissions options, shown when "Emissions" is selected
            html.Div(id='emissions-options', children=[
                html.H4("Emissions Gas Type", style={'margin-bottom': '15px', 'font-size': '18px'}),
                html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '15px'}),
                dcc.Checklist(
                    id='gas-type-checklist',
                    options=[
                        {'label': 'Overall emissions', 'value': 'Overall Emissions'},
                        {'label': 'Carbon Dioxide emissions', 'value': 'CO₂ Emissions'},
                        {'label': 'Nitrous Oxide emissions', 'value': 'N₂O Emissions'},
                        {'label': 'Methane emissions', 'value': 'Methane Emissions'}
                    ],
                    value=['Overall Emissions'],  # Default selection
                    labelStyle={'display': 'block', 'margin': '15px 0', 'color': 'black', 'font-size': '15px'},
                    inline=False
                ),
                html.Div(id='message', style={'color': 'red', 'display': 'none'})
            ]),

            #Projections
            html.H4("Projection", style={'margin-top': "25px", "margin-bottom": "15px", "font-size": "18px"}),
            html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '15px'}),
            dcc.RadioItems(
                id='projection-radio',
                options=[
                    {'label': "Robinson", "value": "robinson"},
                    {'label': "Mollweide", "value": "mollweide"},
                ],
                value="robinson",
                labelStyle={"display": "block", "margin": "15px 0", "color": "black", "font-size": "15px"}
            )
        ], id='collapse-menu'),
    ], style={'width': '22%', 'float': 'left', 'margin-top': '-7%'}),

    #Choropleth map Div
    html.Div([
    dcc.Graph(id='choropleth'),

    # Year Slider Div - Emissions
    html.Div(id='emissions-slider-div', children=[
        dcc.Slider(
            id='year-slider-emissions',
            min=df['Year'].min(),
            max=df['Year'].max(),
            value=df['Year'].max(),
            marks={str(year): '' for year in df['Year'].unique()},
            step=None,
            tooltip={"placement": "bottom", "always_visible": False},
            included=False,
        ),
        
        # Adding Min and Max year labels for Emissions Slider
        html.Div([
            html.Span(str(df['Year'].min()), style={'font-size': '14px', 'position': 'absolute', 'left': '-16px'}),
            html.Span(str(df['Year'].max()), style={'font-size': '14px', 'position': 'absolute', 'right': '-16px'})
        ], style={'position': 'relative', 'width': '100%', 'margin-top': '-40px', 'display': 'flex', 'justify-content': 'space-between'}),
    ], style={'width': '80%', 'margin-left': '1.5%'}),

    # Year Slider Div - Tax status
    html.Div(id='tax-slider-div', children=[
        dcc.Slider(
            id='year-slider-tax',
            min=df_tax['Year'].min(),
            max=df_tax['Year'].max(),
            value=df_tax['Year'].max(),
            marks={str(year): '' for year in df_tax['Year'].unique()},
            step=None,
            tooltip={"placement": "bottom", "always_visible": False},
            included=False
        
        ),
        # Adding Min and Max year labels for Tax Slider
        html.Div([
            html.Span(str(df_tax['Year'].min()), style={'font-size': '14px', 'position': 'absolute', 'left': '-16px'}),
            html.Span(str(df_tax['Year'].max()), style={'font-size': '14px', 'position': 'absolute', 'right': '-16px'})
        ], style={'position': 'relative', 'width': '100%', 'margin-top': '-40px', 'display': 'flex', 'justify-content': 'space-between'}),
    ], style={'width': '80%', 'margin-left': '1.5%'}),

    #Year Slider Div - Temperature
    html.Div(id='temperature-slider-div', children=[
        dcc.Slider(
            id='year-slider-temp',
            min=df_temp['Year'].min(),
            max=df_temp['Year'].max(),
            value=df_temp['Year'].max(),
            marks={str(year): '' for year in df_temp['Year'].unique()},
            step=None,
            tooltip={"placement": "bottom", "always_visible": False},
            included=False,
        ),
        html.Div([
            html.Span(str(df_temp['Year'].min()), style={'font-size': '14px', 'position': 'absolute', 'left': '-16px'}),
            html.Span(str(df_temp['Year'].max()), style={'font-size': '14px', 'position': 'absolute', 'right': '-16px'})
        ], style={'position': 'relative', 'width': '100%', 'margin-top': '-40px', 'display': 'flex', 'justify-content': 'space-between'}),
    ], style={'width': '80%', 'margin-left': '1.5%'}),    

], style={"width": "70%", "float":"left","margin-left":"2%"})], style={'font-family': 'Arial, sans-serif'})


<h4><strong>Step 5 : Callbacks</strong></h4>
Callbacks are the core interactive components of a Plotly Dash application that enable dynamic updates and user interactions:
<ul>
    <li>Connect user inputs to application outputs.</li>
    <li>Enable real-time updates of visualizations and content.</li>
    <li>Handle data filtering and processing based on user selections.</li>
</ul>

Function included here are :
<ol>
    <li>Update Figure Function : Handles map visualization updates based on user selections,Processes emissions data filtering and aggregation,Creates color-coded choropleth maps with custom scales,Manages different display modes (Emissions, Carbon Tax, Temperature).</li>
    <li>Toggle Emissions Options : Controls visibility of UI components based on display mode,Manages gas type selection options,Toggles between different slider controls.</li>
    <li>Collapsible menu toggle Function</li>
    <li>Checklist and Message Handler : Manages gas type selection logic,Handles validation messages,Controls user interface feedback.</li>
</ol>

In [17]:
# Callbacks

#1. Update Figure Function
@app.callback(
    Output('choropleth', 'figure'),
    [Input('year-slider-emissions', 'value'),
     Input('year-slider-tax', 'value'),
     Input('year-slider-temp', 'value'),
     Input('gas-type-checklist', 'value'),
     Input('projection-radio', 'value'),
     Input('display-mode-radio', 'value')]
)
def update_figure(selected_year_emissions, selected_year_tax, selected_year_temp, selected_gas_types, selected_projection, display_mode):
    if display_mode == 'Emissions':
        selected_year = selected_year_emissions
        filtered_df = df[(df['Year'] == selected_year_emissions) & df['Gas Type'].isin(selected_gas_types)]

        if len(selected_gas_types) > 1:
            # Fill NaN values with 0 for relevant columns
            filtered_df['CO₂ Emissions'] = filtered_df['CO₂ Emissions'].fillna(0)
            filtered_df['N₂O Emissions'] = filtered_df['N₂O Emissions'].fillna(0)
            filtered_df['Methane Emissions'] = filtered_df['Methane Emissions'].fillna(0)
            filtered_df['Emissions'] = filtered_df['Emissions'].fillna(0)
        
            # Group by Entity, Code, and Year, and aggregate the relevant columns
            aggregated_df = (
                filtered_df.groupby(['Entity', 'Code', 'Year'], as_index=False)
                .agg({
                    'Emissions': 'sum',
                    'CO₂ Contribution (%)': 'mean',  # Averaging contributions, adjust as needed
                    'N₂O Contribution (%)': 'mean',
                    'Methane Contribution (%)': 'mean',
                    'CO₂ Emissions': 'sum',
                    'N₂O Emissions': 'sum',
                    'Methane Emissions': 'sum'
                })
            )
            filtered_df = aggregated_df
        bins = [0, 1_000_000, 10_000_000, 100_000_000, 1_000_000_000, 10_000_000_000, float('inf')]
        labels = ['0-1 million', '1-10 million', '10-100 million', '100 million - 1 billion', '1-10 billion', '10 billion +']
        color_map = {
            '0-1 million': "#00224c",         # Deep Blue
            '1-10 million': "#264a67", # Bluish Teal
            '10-100 million': "#437792", # Muted Teal
            '100 million - 1 billion': "#5da58b", # Green-Teal
            '1-10 billion': "#b4e06e", # Soft Green
            '10 billion +': "#fce36a", # Pastel Yellow
            'No Data': "#d3d3d3"       # Neutral Gray for 'No Data'
        }
        # Categorize the emissions into bins
        filtered_df['Emission Range'] = pd.cut(filtered_df['Emissions'], bins=bins, labels=labels, include_lowest=True)

        # Handle NaN values by assigning them to 'No Data' category
        filtered_df['Emission Range'] = filtered_df['Emission Range'].cat.add_categories(['No Data'])  # Add 'No Data' as a category
        filtered_df['Emission Range'] = filtered_df['Emission Range'].fillna('No Data')  # Replace NaNs with 'No Data'

        # Convert 'Emission Range' to a categorical type with correct order and include 'No Data'
        filtered_df['Emission Range'] = pd.Categorical(
            filtered_df['Emission Range'],
            categories=labels + ['No Data'],  # Adding 'No Data' category
            ordered=True
        )

        # Sort the DataFrame by the ordered category (optional for clarity)
        filtered_df = filtered_df.sort_values('Emission Range')

        # Set the hover data based on selected gas types and available columns
        hover_data = {
            "Entity": False,
            "Emission Range": True
        }

        # Dynamically add gas-specific columns to hover data if they exist in the selected types and the filtered dataset
        if 'CO₂ Emissions' in selected_gas_types and 'CO₂ Emissions' in filtered_df.columns:
            hover_data["CO₂ Emissions"] = ":,.2f"
        if 'N₂O Emissions' in selected_gas_types and 'N₂O Emissions' in filtered_df.columns:
            hover_data["N₂O Emissions"] = ":,.2f"
        if 'Methane Emissions' in selected_gas_types and 'Methane Emissions' in filtered_df.columns:
            hover_data["Methane Emissions"] = ":,.2f"

        # Handle "Overall Emissions" logic
        if 'Overall Emissions' in selected_gas_types:
            hover_data = {
                "Entity": False,
                "Emissions": ":,.2f",
                "CO₂ Contribution (%)": ":.2f",
                "N₂O Contribution (%)": ":.2f",
                "Methane Contribution (%)": ":.2f"
            }

        # Create choropleth map
        fig = px.choropleth(
            filtered_df,
            locations="Entity",
            locationmode='country names',
            color="Emission Range",  # Color by the custom range
            hover_name="Entity",
            hover_data=hover_data,  # Show the emissions on hover
            color_discrete_map=color_map,
            projection=selected_projection,
            title=f'Emissions in {selected_year} (measured in tonnes of CO₂ equivalents)'
        )

        # Update title alignment to center
        fig.update_layout(
            height=600,  # Increase the height of the map
            width=1000,  # Increase the width of the map
            margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins to reduce gaps
            title_x=0.5,
            title_y=0.95,
            legend=dict(
                orientation="h",  # Horizontal legend
                yanchor="bottom",
                y=-0.1,  # Adjust legend position closer to the map
                xanchor="center",
                x=0.5
            )
        )
    elif display_mode == 'Carbon Tax Status':
        selected_year = selected_year_tax
        filtered_df_tax = df_tax[df_tax['Year'] == selected_year_tax]

        fig = px.choropleth(
            filtered_df_tax,
            locations="Entity",
            locationmode='country names',
            color="tax",
            hover_name="Entity",
            hover_data={
                "Entity": False,
                "tax": True
            },
            color_discrete_map = {
                'No carbon tax': '#F6C2A4',   # Warm coral (absence - subtle, doesn't dominate)
                'Has a carbon tax': '#A2C8E1',  # Bold green (presence - stands out vividly)
                'Has a carbon tax only at a sub-national level': '#D9A6F0',  # mellow yellow 
                'No Data': '#D3D3D3'         # Neutral grey for missing data
            },
            projection=selected_projection,
            title=f'Carbon Tax Status in {selected_year}'
        )

        fig.update_layout(
            height=600,
            width=1000,
            margin=dict(l=50, r=50, t=50, b=50),
            title_x=0.5,
            title_y=0.95,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.1,
                xanchor="center",
                x=0.5
            )
        )
    else:
        selected_year = selected_year_temp
        filtered_df_temp = df_temp[df_temp['Year'] == selected_year_temp]
        

        fig = px.choropleth(
            filtered_df_temp,
            locations="Entity",
            locationmode='country names',
            color="Temperature Change",
            hover_name="Entity",
            hover_data={
                "Entity": False,
                "Temperature Change": True
            },
            color_continuous_scale='Inferno_r',
            projection=selected_projection,
            title=f'Temperature Change caused by emissions in {selected_year} (measured in °C)'
        )

        fig.update_layout(
            height=600,
            width=1150,
            margin=dict(l=50, r=50, t=50, b=50),
            title_x=0.5,
            title_y=0.95,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.1,
                xanchor="center",
                x=0.5
            )
        )

    return fig


#2. Callback to toggle slider visibility based on display mode
@app.callback(
    [Output('emissions-options', 'style'),
     Output('gas-type-checklist', 'options'),
     Output('emissions-slider-div', 'style'),
     Output('tax-slider-div', 'style'),
    Output('temperature-slider-div', 'style')],
    [Input('display-mode-radio', 'value')]
)
def toggle_emissions_options(display_mode):
    if display_mode == 'Emissions':
        # Enable emissions-related options and slider
        options = [
            {'label': 'Overall emissions', 'value': 'Overall Emissions'},
            {'label': 'Carbon Dioxide emissions', 'value': 'CO₂ Emissions'},
            {'label': 'Nitrous Oxide emissions', 'value': 'N₂O Emissions'},
            {'label': 'Methane emissions', 'value': 'Methane Emissions'}
        ]
        return {'display': 'block','width':'98%','margin-left':'2%'}, options, {'display': 'block','width':'90%','margin-left':'2%'}, {'display': 'none'}, {'display': 'none'}
    elif display_mode == 'Carbon Tax Status':
        # Enable tax-related slider
        return {'display': 'none'}, [], {'display': 'none'}, {'display': 'block','width':'90%','margin-left':'2%'}, {'display': 'none'}
    elif display_mode == 'Temperature Change':
        # Enable temperature-related slider
        return {'display': 'none'}, [], {'display': 'none'}, {'display': 'none'}, {'display': 'block','width':'90%','margin-left':'2%'}


#3. Callback to toggle the visibility of the collapsible menu
@app.callback(
    Output('collapse-menu', 'style'),
    [Input('collapse-button', 'n_clicks')]
)
def toggle_collapse(n_clicks):
    if n_clicks % 2 == 0:
        return {'display': 'block', 'margin-top': '48%', 'margin-left': '24%'}
    else:
        return {'display': 'none'}


#4. Callback to handle both checklist logic and message update
@app.callback(
    [Output('gas-type-checklist', 'value'),
     Output('message', 'children'),
     Output('message', 'style')],
    [Input('gas-type-checklist', 'value'),
     Input('display-mode-radio', 'value')]
)
def update_checklist_and_message(selected_values, selected_option):
    try:
        # Handle logic for the gas type checklist
        if 'Overall Emissions' in selected_values and len(selected_values) == 1:
            # If only 'Overall Emissions' is selected, unselect all others
            return ['Overall Emissions'], "", {'display': 'none'}
        elif 'Overall Emissions' == selected_values[-1] and len(selected_values) != 1:
            # If 'Overall Emissions' is last and not the only selection, keep it as the only selected
            return ['Overall Emissions'], "", {'display': 'none'}
        else:
            # Allow multiple selections when 'Overall Emissions' is not involved
            # Show message if "Emissions" is selected and no gas types are selected
            if selected_option == 'emissions' and not selected_values:
                return selected_values, "Please select at least one gas type.", {'color': 'red', 'display': 'block'}
            else:
                return list(set(selected_values) - {'Overall Emissions'}), "", {'display': 'none'}  # Hide message if gases are selected

    except IndexError:
        # If an IndexError occurs, ensure the message is shown
        return selected_values, "Select at least one gas type.", {'color': 'red', 'display': 'block'}




<h4><strong>Step 6 : App Deployment</strong></h4>
This stage handles two key functions:
<ol>
    <li>Browser launch</li>
    <li>Server execution</li>
</ol>

In [18]:
import webbrowser

if __name__ == '__main__':
    # Open the app in a new tab
    webbrowser.open_new("http://127.0.0.1:8051")  # Changed to match the app's port
    app.run_server(debug=True, port=8051)

C:\Users\mkunt\AppData\Local\Temp\ipykernel_19340\1306634367.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mkunt\AppData\Local\Temp\ipykernel_19340\1306634367.py:54: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mkunt\AppData\Local\Temp\ipykernel_19340\1306634367.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa